In [63]:
import pandas as pd
import numpy as np
import math

### Carga dos dados

In [64]:
# definir arquivo Pickle de entrada
#arquivo = 'dados/afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'dados/cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [65]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [66]:
df.shape

(41146, 18)

### Expansão dos dados agregados

In [67]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

41146 1 20389 143
CPU times: user 337 ms, sys: 0 ns, total: 337 ms
Wall time: 335 ms


,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0


In [68]:
# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
df = dfe # apontar para o dataframe expandido

### Uso dos dados originais

In [69]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (82709, 16) (82709,)


In [70]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (55415, 16) (27294, 16) (55415,) (27294,)


In [71]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
THIALLAN,14,1,12,12,1,9,8,20,0,0,0,0,0,0,0,0
FLAVIANE,5,14,1,9,22,1,12,6,0,0,0,0,0,0,0,0
CLAUDERINO,15,14,9,18,5,4,21,1,12,3,0,0,0,0,0,0
EDUARDO,15,4,18,1,21,4,5,0,0,0,0,0,0,0,0,0
ALEXANDRO,15,18,4,14,1,24,5,12,1,0,0,0,0,0,0,0


In [72]:
y_train.head()

NOME
THIALLAN      1
FLAVIANE      0
CLAUDERINO    1
EDUARDO       1
ALEXANDRO     1
Name: SEXO, dtype: uint8

In [73]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import BinaryAccuracy

In [91]:
# determine the number of input features
n_features = X_train.shape[1]

# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))

# summarize the model
model.summary()

# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.8), loss='binary_crossentropy')
model.compile(optimizer='sgd', loss='mse', metrics=[BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)])
#model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_25 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 9         
Total params: 267
Trainable params: 267
Non-trainable params: 0
_________________________________________________________________


In [92]:
%%time

# fit the model
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0)
model

CPU times: user 2min 30s, sys: 10.3 s, total: 2min 40s
Wall time: 1min 41s


In [87]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Acurácia: %.2f %%' % (acc * 100))

Acurácia: 97.31 %


In [88]:
# save model to file
model.save('modelos/modelo-genero-nome-tensorflow.h5')

In [89]:
df2 = X_test.copy()
df2['PROB'] = model.predict(X_test)
df2['REAL'] = y_test
df2['PREV'] = df2['PROB'].apply(lambda x: 1 if x > 0.5 else 0)
df2['DIF'] = df2[['REAL', 'PREV']].apply(lambda x: abs(x[0] - x[1]), axis=1)
#df2['REAL'].apply(lambda x: 'F' if x else 'M')
#df2['PREV'].apply(lambda x: 'F' if x else 'M')
df2.head(15)

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,PROB,REAL,PREV,DIF
NOME,,,,,,,,,,,,,,,,,,,,
LICIO,15,9,3,9,12,0,0,0,0,0,0,0,0,0,0,0,0.996008,1,1,0
SOLANGE,5,7,14,1,12,15,19,0,0,0,0,0,0,0,0,0,0.020247,0,0,0
BERTHA,1,8,20,18,5,2,0,0,0,0,0,0,0,0,0,0,0.006376,0,0,0
VALDENILDA,1,4,12,9,14,5,4,12,1,22,0,0,0,0,0,0,0.000948,0,0,0
JUVANETE,5,20,5,14,1,22,21,10,0,0,0,0,0,0,0,0,0.111323,0,0,0
THAISA,1,19,9,1,8,20,0,0,0,0,0,0,0,0,0,0,0.000266,0,0,0
OLINTO,15,20,14,9,12,15,0,0,0,0,0,0,0,0,0,0,0.996614,1,1,0
TRICIA,1,9,3,9,18,20,0,0,0,0,0,0,0,0,0,0,0.000798,0,0,0
SANANE,5,14,1,14,1,19,0,0,0,0,0,0,0,0,0,0,0.309521,0,0,0


In [90]:
df2[df2['DIF'] > 0].head(15)

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,PROB,REAL,PREV,DIF
NOME,,,,,,,,,,,,,,,,,,,,
WALENBERG,7,18,5,2,14,5,12,1,23,0,0,0,0,0,0,0,0.373637,1,0,1
CALIOPE,5,16,15,9,12,1,3,0,0,0,0,0,0,0,0,0,0.106952,1,0,1
PHILIPPE,5,16,16,9,12,9,8,16,0,0,0,0,0,0,0,0,0.083486,1,0,1
JOSEPH,8,16,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0.045520,1,0,1
GUILHERME,5,13,18,5,8,12,9,21,7,0,0,0,0,0,0,0,0.133713,1,0,1
ESTHER,18,5,8,20,19,5,0,0,0,0,0,0,0,0,0,0,0.945608,0,1,1
QUETI,9,20,5,21,17,0,0,0,0,0,0,0,0,0,0,0,0.735387,0,1,1
JASPE,5,16,19,1,10,0,0,0,0,0,0,0,0,0,0,0,0.169505,1,0,1
ROSE,5,19,15,18,0,0,0,0,0,0,0,0,0,0,0,0,0.741028,0,1,1
